In [1]:
%load_ext autoreload
%autoreload 2 

In [2]:
import numpy as np
import pickle as pk
from jax import numpy as jnp
from jax import random as rnd
from flax.traverse_util import flatten_dict

import torch 

from hwat_jax import FermiNet
from hwat_func import Ansatz_fb

from pyfig import Pyfig
from utils import flat_any
from optree import tree_flatten

In [3]:
# swap params
v_tr = pk.load(open('v_tr.pk', 'rb'))
v_params = v_tr['params']
v_grads = v_tr['grads']
r = v_tr['r']
# v_r = np.random.normal(0, 1, (n_b, n_e, 3))
print(v_tr.keys(), r.shape, v_params.keys(), r[0].shape)

dict_keys(['e', 'grads', 'ke', 'params', 'pe', 'r']) (1, 512, 4, 3) frozen_dict_keys(['params']) (512, 4, 3)


In [4]:
rng = rnd.PRNGKey(1)
cj = Pyfig(wb_mode='disabled', notebook=True)
jmod = cj.partial(FermiNet)
_jp = jmod.init(rng, r[0, [0]])
jp = v_params
for (k, p), (_k, _p) in zip(flatten_dict(jp).items(), flatten_dict(_jp).items()):
    print(k, p.shape, _k, _p.shape)
log_psi = jmod.apply(_jp, r[0])
print(log_psi[:2])
print(v_params['params'].keys(), _jp['params'].keys())

2022-12-27 13:28:07.061537: E external/org_tensorflow/tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


init PlugIn classes
updating configuration
Run: ['git', 'log', '--pretty=format:%h', '-n', '1'] at /home/amawi/projects/hwat
stdout: b90fa37 stderr: 
Run: ['hostname'] at .
stdout: oceanus  stderr: 
running script
setting exp_path
Run: ['git', 'log', '--pretty=format:%h', '-n', '1'] at /home/amawi/projects/hwat
stdout: b90fa37 stderr: 
Run: ['hostname'] at .
stdout: oceanus  stderr: 


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Run: ['git', 'log', '--pretty=format:%h', '-n', '1'] at /home/amawi/projects/hwat
stdout: b90fa37 stderr: 
Run: ['hostname'] at .
stdout: oceanus  stderr: 
('params', 'Dense_0', 'bias') (1, 32) ('params', 'Dense_0', 'bias') (32,)
('params', 'Dense_0', 'kernel') (1, 20, 32) ('params', 'Dense_0', 'kernel') (20, 32)
('params', 'Dense_1', 'bias') (1, 32) ('params', 'Dense_1', 'bias') (16,)
('params', 'Dense_1', 'kernel') (1, 4, 32) ('params', 'Dense_1', 'kernel') (4, 16)
('params', 'Dense_2', 'bias') (1, 32) ('params', 'Dense_2', 'bias') (32,)
('params', 'Dense_2', 'kernel') (1, 160, 32) ('params', 'Dense_2', 'kernel') (128, 32)
('params', 'Dense_3', 'bias') (1, 16) ('params', 'Dense_3', 'bias') (16,)
('params', 'Dense_3', 'kernel') (1, 32, 16) ('params', 'Dense_3', 'kernel') (32, 16)
('params', 'Dense_4', 'bias') (1, 16) ('params', 'Dense_4', 'bias') (16,)
('params', 'Dense_4', 'kernel') (1, 32, 16) ('params', 'Dense_4', 'kernel') (32, 16)
('params', 'Dense_5', 'bias') (1, 2) ('params', '

In [7]:
list(flatten_dict(_jp).items())[0]

(('params', 'Dense_0', 'bias'),
 Array([0.00593489, 0.00268932, 0.00064195, 0.00706287, 0.00621831,
        0.00998503, 0.00726382, 0.00146654, 0.00408086, 0.00807343,
        0.00637496, 0.00919925, 0.00673006, 0.00812667, 0.00376891,
        0.00985797, 0.00689847, 0.00966867, 0.00270503, 0.00158928,
        0.00351574, 0.00832261, 0.00841896, 0.00154599, 0.00474995,
        0.00800335, 0.00841118, 0.00508559, 0.00370565, 0.00682438,
        0.00971892, 0.00518902], dtype=float32))

In [ ]:

tdtype = torch.float64
c = Pyfig(wb_mode='disabled', notebook=True)
c._convert(device='cpu', dtype=tdtype)
n_b, n_e = c.data.n_b, c.data.n_e
torch.set_default_dtype(tdtype)
tr = torch.tensor(r, dtype=tdtype)

tmod = c.partial(Ansatz_fb).to(tdtype)
tp = list(tmod.named_parameters())

In [ ]:


def swap_params(jp, tp):
    jp_flat = flatten_dict(jp)
    print(len(jp_flat), len(tp))
    jp_b = [(k, jp0) for (k, jp0) in  jp_flat.items() if 'bias' in k]
    tp_b = [(k, jp0) for (k, jp0) in  tp if 'bias' in k]
    jp_p = [(k, jp0) for (k, jp0) in  jp_flat.items() if 'kernel' in k]
    tp_p = [(k, jp0) for (k, jp0) in  tp if 'weight' in k]

    order = [0, 2, 1] + [i for i in range(3, 9)]
    key = {jp_b[i][0]:tp_b[j][0] for j, i in enumerate(order)} | {jp_p[i][0]:tp_p[j][0] for j, i in enumerate(order)}
    jp_b = [jp_b[i] for i in order]
    jp_p = [jp_p[i] for i in order]
    new_tp = {tp_b[i][0] : np.array(b[1]) for i,b in enumerate(jp_b)} | {tp_p[i][0] : np.array(p[1]).T for i,p in enumerate(jp_p)}

    # print(len(jp_flat), len(tp))
    # for (k, jp0), (tpk, tp0) in zip(jp_b, tp_b):
    #     print('.'.join(k).replace('kernel', 'weight'), tpk, jp0.shape, tp0.shape)
        
    # print(len(jp_flat), len(tp))
    # for (k, jp0), (tpk, tp0) in zip(jp_p, tp_p):
    #     print('.'.join(k).replace('kernel', 'weight'), tpk, jp0.shape, tp0.shape)
    return new_tp, key

new_tp, key = swap_params(jp, tp)
for name, param in tmod.named_parameters():
    # print(name)
    # print(param.data.dtype, param.data.shape)
    param.data = torch.tensor(new_tp[name], requires_grad=True, dtype=tdtype)
    # print(param.data.dtype, param.data.shape)